In [4]:
# ✅ Step 1: Install and import dependencies
!pip install -q tensorflow plotly
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from google.colab import drive

# ✅ Step 2: Mount Google Drive
drive.mount('/content/drive')

# ✅ Step 3: Load model
model_path = '/content/drive/MyDrive/RNA-peptide/rna_peptide_dual_model_with_4486.keras'
model = load_model(model_path)
intermediate_model = Model(inputs=model.input, outputs=model.get_layer('interaction_features').output)
print("✅ Model loaded")

# ✅ Step 4: Define vocabularies and encoding
rna_vocab = list("AUGC")
aa_vocab = list("ACDEFGHIKLMNPQRSTVWY")
rna_dict = {ch: i for i, ch in enumerate(rna_vocab)}
aa_dict = {ch: i for i, ch in enumerate(aa_vocab)}

def one_hot_encode(seq, vocab_dict, max_len):
    encoded = np.zeros((max_len, len(vocab_dict)))
    for i, ch in enumerate(seq[:max_len]):
        if ch in vocab_dict:
            encoded[i, vocab_dict[ch]] = 1
    return encoded.flatten()

MAX_RNA_LEN = 100
MAX_PEP_LEN = 100

# ✅ Step 5: Load predict file (CSV)
predict_df = pd.read_csv('/content/drive/MyDrive/RNA-peptide/predict.csv')  # Make sure it has `rna_seq` and `protein_seq` columns

# ✅ Step 6: Loop through each row and plot both static and interactive heatmaps
for idx, row in predict_df.iterrows():
    rna_seq = row['rna_seq']
    pep_seq = row['protein_seq']

    encoded_rna_input = np.expand_dims(one_hot_encode(rna_seq, rna_dict, MAX_RNA_LEN), axis=0)
    encoded_peptide_input = np.expand_dims(one_hot_encode(pep_seq, aa_dict, MAX_PEP_LEN), axis=0)

    interaction_features = intermediate_model.predict([encoded_rna_input, encoded_peptide_input])[0]
    interaction_matrix = interaction_features.reshape(8, 8)

    # ✅ Create biologically meaningful labels
    rna_labels = [f"{base}{i+1}" for i, base in enumerate(rna_seq[:8])]
    pep_labels = [f"{aa}{i+1}" for i, aa in enumerate(pep_seq[:8])]

    # ✅ Plot static heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(interaction_matrix, xticklabels=pep_labels, yticklabels=rna_labels,
                cmap='viridis', annot=True, fmt=".2f", linewidths=0.5,
                cbar_kws={'label': 'Activation'})
    plt.title(f"RNA–Peptide Interaction (Sample {idx})")
    plt.xlabel("Peptide Positions")
    plt.ylabel("RNA Bases")
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/RNA-peptide/heatmap_sample_{idx}.png")
    plt.close()

    # ✅ Interactive heatmap
    df_matrix = pd.DataFrame(interaction_matrix, index=rna_labels, columns=pep_labels)
    fig = px.imshow(df_matrix, text_auto=True,
                    labels=dict(x="Peptide", y="RNA", color="Activation"),
                    title=f"Interactive RNA–Peptide Heatmap (Sample {idx})",
                    color_continuous_scale="Viridis")
    fig.write_html(f"/content/drive/MyDrive/RNA-peptide/interactive_heatmap_sample_{idx}.html")
    print(f"✅ Saved heatmaps for Sample {idx}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model loaded
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
✅ Saved heatmaps for Sample 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
✅ Saved heatmaps for Sample 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
✅ Saved heatmaps for Sample 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
✅ Saved heatmaps for Sample 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
✅ Saved heatmaps for Sample 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
✅ Saved heatmaps for Sample 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
✅ Saved heatmaps for Sample 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
✅ Saved heatmaps for Sample 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
✅ Saved heatmaps for Sample 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
✅ Saved heatmaps for Sample 9
